In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from video_processor import VideoProcessor
import datetime
import os


In [2]:
# import os
# import pandas as pd

# DATASET_DIR = '../datasets/JHMDB/'

# # Function to get list of video clips and their labels
# def get_video_clips_and_labels(dataset_dir):
#     data = {'clip_path': [], 'label': []}
    
#     for root, dirs, files in os.walk(dataset_dir):
#         for dir_name in dirs:
#             label = dir_name
#             clips_dir = os.path.join(root, dir_name)
            
#             for clip_file in os.listdir(clips_dir):
#                 clip_path = os.path.join(clips_dir, clip_file)
#                 data['clip_path'].append(clip_path)
#                 data['label'].append(label)
    
#     return data

# # Create DataFrame and save to CSV
# data = get_video_clips_and_labels(DATASET_DIR)
# df = pd.DataFrame(data)
# csv_path = os.path.join(DATASET_DIR, 'dataset.csv')
# df.to_csv(csv_path, index=False)

# print(f"CSV file saved at: {csv_path}")


In [3]:
df = pd.read_csv('../datasets/JHMDB/dataset.csv')
df.head()

,clip_path,label
0,../datasets/JHMDB/brush_hair\April_09_brush_ha...,brush_hair
1,../datasets/JHMDB/brush_hair\April_09_brush_ha...,brush_hair
2,../datasets/JHMDB/brush_hair\April_09_brush_ha...,brush_hair
3,../datasets/JHMDB/brush_hair\Aussie_Brunette_B...,brush_hair
4,../datasets/JHMDB/brush_hair\Aussie_Brunette_B...,brush_hair


In [4]:
df['label'].unique()

array(['brush_hair', 'catch', 'clap', 'climb_stairs', 'golf', 'jump', 'kick_ball', 'pick', 'pour', 'pullup', 'push', 'run', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'stand', 'swing_baseball', 'throw', 'walk', 'wave'], dtype=object)

# Inference on Dataset

In [5]:
from ultralytics import YOLO
import cv2

In [6]:
model = YOLO('models/yolov8s-pose.pt') 

In [8]:
batch_size = 10
vp = VideoProcessor(model,batch_size, max_frames=10)

keypoints_df = pd.DataFrame()
class_keypoints_df = pd.DataFrame()
for train_df in [df]:  #, punch_df[0:1]
    # label = train_df['label'][0]
    # print(f"Training: {label}")
    batch, skipped_id = vp.process_video_batch(train_df['clip_path'])
    batch = np.array(batch)[:,0,:].reshape(-1,batch_size , 34)
    class_keypoints_df = pd.DataFrame({ 'keypoints': batch.tolist(), 'label': train_df['label'].drop(skipped_id)})
    keypoints_df = pd.concat([keypoints_df, class_keypoints_df])
    print(f'Skipped: {len(skipped_id)}')

# keypoints_df.columns = ['keypoints', 'label']

cv2.destroyAllWindows()

Processing Videos: 100%|██████████| 929/929 [06:19<00:00,  2.45it/s, skipped=Too short! (Skipped: 1)]


ValueError: array length 928 does not match index length 929

In [20]:
keypoints_df

,keypoints,label
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.54...",brush_hair
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",brush_hair
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.55...",brush_hair
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",brush_hair
4,"[[0.48466691970825193, 0.33056793212890623, 0....",brush_hair
...,...,...
924,"[[0.49116811752319334, 0.39432493845621747, 0....",wave
925,"[[0.5412163734436035, 0.44961668650309244, 0.5...",wave
926,"[[0.3128415822982788, 0.3677842458089193, 0.34...",wave
927,"[[0.612129545211792, 0.5730491638183594, 0.672...",wave


In [19]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'keypoint_dataset/JHMDB/keypoints_df_{timestamp}.csv'

# Save the keypoints_df to the new file
keypoints_df.to_csv(filename, index=False)

In [24]:
np.array(keypoints_df['keypoints'].tolist()).shape

(928, 10, 34)